In [1]:
from dqanneal.graph.embed import (get_problem_graph, 
                                  embed_problem_onto_hardware, 
                                  define_embedded_qubo_problem,
                                  unembed_samples)
from dqanneal.solvers.solvers import dimod_optimizer_local, save_annealing_results
import dimod

In [2]:
dwave_qubo = {
    (0, 0):  -1,
    (0, 1):   2,
    (0, 2):  -3,
    (0, 3):   4,
    (0, 4):  -5,
    (0, 5):   6,
    (0, 6):  -7,
    (0, 7):   8,
    (0, 8):  -9,
    (0, 9):   10,
    (0, 10): -11,
    (0, 11):  12,
    (0, 12): -13,
    (0, 13):  14,
    (0, 14): -15,
    (0, 15):  16,
    (0, 16): -17,
    (0, 17):  18,
    (0, 18): -19,
    (0, 19):  20,
    (0, 20): -21,
    (4, 16):  22,
    (1, 19): -23,
    }

In [3]:
source_bqm = dimod.BinaryQuadraticModel.from_qubo(dwave_qubo)
print(f'number of variables: {len(source_bqm.variables)}')

number of variables: 21


In [4]:
problem_graph, problem_edge_dict = get_problem_graph(dwave_qubo)

In [5]:
### need to setup dwave leap token access for this

# from dwave.system.samplers import DWaveSampler
# import networkx as nx
# sampler_dw = DWaveSampler(solver={'topology__type': 'pegasus'})
# G = nx.from_dict_of_lists(sampler_dw.adjacency)
# edge_dict = {node: set(G.neighbors(node)) for node in G.nodes}

In [6]:
import networkx as nx
import dwave_networkx as dnx

# https://docs.ocean.dwavesys.com/en/latest/docs_dnx/reference/generators.html
# G = dnx.chimera_graph(1, 1, 4)
# G = dnx.zephyr_graph(6)

# https://github.com/dwave-examples/pegasus-notebook/blob/master/01-exploring-pegasus.ipynb
hardware_graph = dnx.pegasus_graph(16)
hardware_graph_edge_dict = {node: set(hardware_graph.neighbors(node)) for node in hardware_graph.nodes}

In [7]:
embedded_problem, valid_flag = embed_problem_onto_hardware(problem_graph,
                                                            hardware_graph,
                                                            maxtime_sec=30,
                                                            attempts=10)

In [8]:
print(f'Was successful: {bool(valid_flag)}')

Was successful: True


In [9]:
embedded_problem

{0: [5552, 539],
 1: [524],
 2: [5627],
 3: [569],
 4: [5567],
 5: [5507],
 6: [5551],
 7: [629],
 8: [554],
 9: [5537],
 10: [644],
 11: [5582],
 12: [659],
 13: [5522],
 14: [584],
 15: [689],
 16: [614],
 17: [5657],
 18: [538],
 19: [5597],
 20: [5492]}

In [10]:
embedded_qubo, convert_fn_sample = define_embedded_qubo_problem(
                                    embedded_problem,
                                    hardware_graph_edge_dict,
                                    dwave_qubo)

In [11]:
embedded_bqm = dimod.BinaryQuadraticModel.from_qubo(embedded_qubo)
print(f'number of variables: {len(embedded_bqm.variables)}')

number of variables: 22


In [12]:
optimizer = dimod_optimizer_local(optimizer_type='ExactSolver')
embedded_sampleset, embedded_best_sample = optimizer.sample_qubo(embedded_qubo, 500)

/Users/lex/anaconda3/envs/dwave310/lib/python3.10/site-packages/dimod/core/sampler.py:292: SamplerUnknownArgWarning: Ignoring unknown kwarg: 'num_reads'
  return self.sample(bqm, **parameters)


In [13]:
embedded_sampleset

SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   0.        , 1),
           ([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  56.88222359, 1),
           ([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  -1.        , 1),
           ...,
           ([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], -22.        , 1),
           ([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  35.88222359, 1),
           ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   0.        , 1)],
          dtype=[('sample', 'i1', (22,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables([524, 538, 539, 554, 569, 584, 614, 629, 644, 659, 689, 5492, 5507, 5522, 5537, 5551, 5552, 5567, 5582, 5597, 5627, 5657]), {}, 'BINARY')

In [14]:
converted_samples, converted_best_sample = unembed_samples(embedded_sampleset,
                                    embedded_problem, 
                                    source_bqm)
converted_samples

SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   0., 1),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  -1., 1),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  -1., 1),
           ...,
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], -22., 1),
           ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], -22., 1),
           ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],   0., 1)],
          dtype=[('sample', 'i1', (21,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(range(0, 21)), {}, 'BINARY')

In [15]:
solver = dimod.ExactSolver()
out = dimod.ExactSolver().sample_qubo(dwave_qubo)
source_bqm.energy(out.first.sample)

np.float64(-117.0)

In [16]:
embedded_bqm.energy(embedded_best_sample)

np.float64(-117.0)

In [19]:
unembedded_samples, unembedded_best = unembed_samples(embedded_sampleset,
                                    embedded_problem, 
                                    source_bqm)

In [20]:
source_bqm.energy(unembedded_best)

np.float64(-117.0)

In [21]:
# dimod.BinaryQuadraticModel.from_serializable(source_bqm.to_serializable())

In [22]:
extra_dictionary = {
    'source_bqm':source_bqm.to_serializable(),
    'embedded_problem': embedded_problem
}

In [24]:
output_file_path = save_annealing_results(embedded_sampleset,
                        extra_ending='test',
                        verbose=False,
                        extra_dictionary=extra_dictionary)